In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df=spark.read.format("parquet").load("abfss://bronze@databricksstorageak.dfs.core.windows.net/products")

In [0]:
df=df.drop("_rescued_data")

In [0]:
df.createOrReplaceTempView("products")


In [0]:
%sql
CREATE OR REPLACE FUNCTION  catabricks_cat.bronze.discount_func(p_price double)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
select product_id,catabricks_cat.bronze.discount_func(price) as discounted_price from products

In [0]:
df=df.withColumn("discounter_price",expr("catabricks_cat.bronze.discount_func(price)"))


In [0]:
%sql
 create or replace function catabricks_cat.bronze.upper_func(brand string)
 returns string
 language sql
 return upper(brand)

In [0]:
%sql
select brand,catabricks_cat.bronze.upper_func(brand) as upper_brand from products

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksstorageak.dfs.core.windows.net/products")

In [0]:
%sql
create table if not exists catabricks_cat.silver.products_silver using delta 
location 'abfss://silver@databricksstorageak.dfs.core.windows.net/products'